## Scraping MLS Player Stats

Scraping www.mlssoccer.com/ for player statistics

### Import Necessary Modules

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime 
from dateutil import parser
import os
import time
import itertools
import re
import pickle

### Scrape Statistics

#### Define Scraper and Helpers

In [3]:
def send_request(url, params={}, attempts=3):

    cnt = 0
    while cnt <= attempts:
        cnt += 1        
        # Send Request
        response = requests.get(url, params)
        # Check Status
        print(response.url, '\n', response.status_code, response.reason)
        if (response.status_code >= 200) and (response.status_code < 300):           
            return response
        # If bad status, pause before trying again
        print('Pause, then retry')
        time.sleep(15)
    return None

In [4]:
def map_id_to_club(franchise_id):
    
    franchise_map = {11091: 'ATL',
                     1207: 'CHI',
                     436: 'COL',
                     454: 'CLB',
                     1326: 'DC',
                     1903: 'DAL',
                     1897: 'HOU',
                     1230: 'LA',
                     11690: 'LAFC',
                     6977: 'MNUFC',
                     1616: 'MTL',
                     928: 'NE',
                     9668: 'NYCFC',
                     399: 'NYRB',
                     6900: 'ORL',
                     5513: 'PHI',
                     1581: 'POR',
                     1899: 'RSL',
                     1131: 'SJ',
                     3500: 'SEA',
                     421: 'KC',
                     2077: 'TOR',
                     1708: 'VAN'}
    return franchise_map[franchise_id]
        

In [5]:
def check_for_data(stats_table):
    # Check if the table has any 
    odd_rows = stats_table.findAll('tr', {'class': 'odd'})
    if odd_rows[0].text.strip() == 'Stats Unavailable':
        return False
    return True    

In [6]:
def split_player_name(df):
    first = []
    last = []
    for t in df.Player.str.split():
        if len(t) == 1:
            first.append(t[0])
            last.append('')
        elif len(t) == 2:
            first.append(t[0])
            last.append(t[1])
        else:
            first.append(t[0])
            last.append(' '.join(t[1:]))
    
    df['Last'] = last
    df['First'] = first  
    
    df = df.drop('Player', axis=1)
    
    return df
        

In [7]:
# # Find and parse franchise popup menu to get list of teams
# soup = BeautifulSoup(response.text, 'lxml')
# franchise_select = soup.find('select', {'id': 'edit-franchise', 'name': 'franchise'})
# franchise = []
# # start with second index since first is generic "Select A Club"
# for team in franchise_select.findAll('option')[1:]:
#     franchise.append(int(team['value']))

In [8]:
def generate_param_combos(param_groups):
    params = []
    for combo in itertools.product(*param_groups):
        params.append({'franchise': combo[0],
                       'group': combo[1],
                       'season_type': combo[2],
                       'year': combo[3],
                       'page': combo[4]})
    return params

In [9]:
def scrape_player_stats(base_url, params, return_last_pg=False):

    response = send_request(base_url, params)
    if response is None:
        return None
    
    # Parse HTML
    soup = BeautifulSoup(response.text,'lxml')
    
    stats_table = soup.find('table')
    
    if not check_for_data(stats_table):
        return None
    
    stats_df = extract_stats(stats_table)
    # Add Year Column
    stats_df['Year'] = np.repeat(int(params['year']), len(stats_df))
    # Add Season Column
    stats_df['Season'] = np.repeat(params['season_type'], len(stats_df))
    
    if return_last_pg:
        return stats_df, get_last_page(soup)
    
    return stats_df     

In [10]:
def extract_stats(stat_table):
    
    # Extract salary data
    stat_header = []
    stat_data = []
    for row in stat_table.findAll('tr'):
        row_data = []

        # Get row type and check if header or data row
        row_type = row.findChild().name
        if row_type == 'th':
            # Extract header
            for h in stat_table.findAll('th'):
                stat_header.append(h.text) 
        else:
            # Extract data
            for data in row.findAll('td'):
                row_data.append(data.text)
            stat_data.append(row_data)    
            
    # Compile stat dataframe
    stat_df = pd.DataFrame(stat_data, columns=stat_header)
    
    # Strip any whitespace from column names
    stat_df.columns = stat_df.columns.str.strip()
    
    # Replace blanks with nans
    stat_df = stat_df.replace('', np.nan)
        
    return stat_df

In [11]:
def get_last_page(soup):

    last_pg_url = soup.find('li', {'class': 'pager-last last'}).find('a')['href']
    last_pg = int(re.search('(?<=page=)\d+', last_pg_url).group())
    
    return last_pg

In [12]:
def scrape_group_stats(base_url, group, year=np.arange(2007, 2018), franchise='select', season_type='REG'):
    
    group_df = pd.DataFrame()
    combos = generate_param_combos([[franchise], [group], [season_type], year, [0]])
    for params in combos: 
        stats_df, last_pg = scrape_player_stats(base_url, params, True)
        
        if stats_df is None:
            continue
            
        # Scrape first page of results
        group_df = pd.concat([group_df, stats_df], axis=0)
        # Add pause to prevent 429 status
        print(datetime.now())
        time.sleep(np.random.uniform(2, 5)) 
        
        # Scrape next through last
        for idx in range(1, last_pg+1):
            params['page'] = idx
            stats_df = scrape_player_stats(base_url, params)
            if stats_df is None:
                continue
            group_df = pd.concat([group_df, stats_df], axis=0)            
            # Add pause to prevent 429 status
            print(datetime.now())
            time.sleep(np.random.uniform(2, 5)) 
            
    # Reset index to remove duplicates created during concatenation
    group_df.reset_index(inplace=True, drop=True)
    
    return group_df        

#### Send Request to Base URL and Verify Site is Up

In [13]:
base_url = 'http://www.mlssoccer.com/stats/season'
response = requests.get(base_url)
response.status_code, response.url

(200, 'https://www.mlssoccer.com/stats/season')

#### Define parameter options

In [ ]:
# group = ['goals', 'assists', 'shots', 'fouls', 'goalkeeping']
# for grp in group:
#     print('Scraping:', grp, datetime.now(), '\n')
#     df = scrape_group_stats(base_url, grp, np.arange(2007, 2018, 1))
#     df.to_pickle(grp + '_df.pkl')
#     print('Completed Scraping:', grp)

In [ ]:
fouls_df = scrape_group_stats(base_url, 'fouls')

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=0 
 200 OK
2018-01-26 17:36:43.348369
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=1 
 200 OK
2018-01-26 17:36:53.232375
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=2 
 200 OK
2018-01-26 17:37:04.300632
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=3 
 200 OK
2018-01-26 17:37:18.309698
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=4 
 200 OK
2018-01-26 17:37:30.313162
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=5 
 200 OK
2018-01-26 17:37:38.724534
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=6 
 200 OK
2018-01-26 17:37:45.731797
https://www.mlssoccer.com/stats/season?fr

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=13 
 200 OK
2018-01-26 17:45:10.260328
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=14 
 200 OK
2018-01-26 17:45:16.394818
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=15 
 200 OK
2018-01-26 17:45:23.879019
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=16 
 200 OK
2018-01-26 17:45:33.090253
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2011&page=0 
 200 OK
2018-01-26 17:45:39.355003
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2011&page=1 
 200 OK
2018-01-26 17:45:47.372754
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2011&page=2 
 200 OK
2018-01-26 17:45:55.433108
https://www.mlssoccer.com/stats/seaso

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=14 
 200 OK
2018-01-26 17:55:06.438992
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=15 
 200 OK
2018-01-26 17:55:18.458530
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=16 
 200 OK
2018-01-26 17:55:28.057691
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=17 
 200 OK
2018-01-26 17:55:44.132065
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=18 
 200 OK
2018-01-26 17:55:54.680042
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=19 
 200 OK
2018-01-26 17:56:06.061365
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=20 
 200 OK
2018-01-26 17:56:18.816256
https://www.mlssoccer.com/stats/se

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=8 
 200 OK
2018-01-26 18:04:58.745835
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=9 
 200 OK
2018-01-26 18:05:08.883285
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=10 
 200 OK
2018-01-26 18:05:15.896533
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=11 
 200 OK
2018-01-26 18:05:24.135915
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=12 
 200 OK
2018-01-26 18:05:30.177516
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=13 
 200 OK
2018-01-26 18:05:37.388959
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=14 
 200 OK
2018-01-26 18:05:47.682119
https://www.mlssoccer.com/stats/seas

In [ ]:
fouls_df.info()

In [ ]:
fouls_df

In [ ]:
fouls_df.to_pickle('fouls_df.pkl')

### Sandbox